In [1]:
#from selenium import webdriver
#from selenium.webdriver.support.wait import WebDriverWait
#from selenium.webdriver.support import expected_conditions
#from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.action_chains import ActionChains
#from selenium.webdriver.support.ui import Select

import pandas as pd
import pickle
import os
import sys
import warnings
import time

from bs4 import BeautifulSoup
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


#from selenium.webdriver.edge.options import Options

#edge_options = Options()
#edge_options.add_argument('--headless')
#edge_options.add_argument("--disable-images")  # Disable images
#edge_options.add_argument("--disable-javascript")  # Disable JavaScript
#edge_options.add_argument("--disable-plugins")  # Disable plugins
#edge_options.add_argument("--disable-extensions")  # Disable extensions
#edge_options.add_argument("--blink-settings=imagesEnabled=false")  # Another image disabling flag


# Set a minimal User-Agent (like a text-only browser)
#edge_options.add_argument("user-agent=Links (2.25; Linux 2.4.24)")

#driver = webdriver.Edge(options=edge_options)
#driver.execute_cdp_cmd("Network.setBlockedURLs", {"urls": ["*.jpg", "*.png", "*.gif","*.svg"]})
#driver.execute_cdp_cmd("Network.enable", {})


#driver.maximize_window()

#driver.execute_script("document.body.style.zoom='67%'")

driver.get('https://www.olympedia.org/athletes/advanced_search')

sport_codes={e.get_attribute('value'):e.text for e in driver.find_elements(By.CSS_SELECTOR, 'select[name="sport_group"][id="sport_select"] option') if e.text!=''}

sport_athletes={}

for i, (sport_code, sport) in enumerate(sport_codes.items(),start=1):
    all_athletes=[]
    for page in range(1,10001):
        url=f'https://www.olympedia.org/athletes/search_athlete?affiliations=&bio=&birthcountry=&birthplace=&competition_type_id=&continent=&country=&date%5Bbirthday%5D=&date%5Bbirthmonth%5D=&date%5Bbirthyear%5D=&date%5Bdeathday%5D=&date%5Bdeathmonth%5D=&date%5Bdeathyear%5D=&deathcountry=&deathplace=&event_name=&gender=&given_name=&id=&image=&medal=&page={page}&second_country=&sport=&sport_group={sport_code}&surname=&year2=&year='
        html=requests.get(url,headers).content
        soup=BeautifulSoup(html,'html.parser')
        athletes=['https://www.olympedia.org'+e['href'] for e in soup.select('table[class="table table-striped"] tr td a') if 'athletes' in e['href']]
        if len(athletes)==0:
            break
        all_athletes.extend(athletes)
        
        print(f'Sport: {i}/{len(sport_codes)}, Page: {page}, Total: {len(all_athletes)}'.ljust(80), end='\r')
    
    sport_athletes[sport]=all_athletes
    


In [2]:
#pickle.dump(sport_athletes, open('sport_athletes.pickle','wb'))
sport_athletes=pickle.load( open('sport_athletes.pickle','rb'))

In [7]:
total = len([url for sublist in sport_athletes.values() for url in sublist])
total

181059

In [4]:
sorted_files=sorted(os.listdir('scraped_athletes'), key=lambda x: (int(x.split('_')[2]), int(x.split('_')[3].split('.pickle')[0])))

max_scraped={}
max_sport_index=0
for file in sorted_files:
    sport_index=int(file.split('_')[2])
    athlete_index=int(file.split('_')[3].split('.pickle')[0])
    max_scraped[sport_index]=athlete_index

    if sport_index>max_sport_index:
        max_sport_index=sport_index
print(max_scraped)
print(max_sport_index)

{1: 2239, 2: 46, 3: 820, 4: 16135, 5: 1374, 6: 86, 7: 27331, 8: 38, 9: 1252, 10: 5005, 11: 13, 17: 3602, 18: 36, 19: 80, 20: 10, 22: 6703, 23: 24, 24: 13, 25: 2605, 26: 10477, 27: 5196, 28: 66, 29: 12, 30: 10, 31: 474, 34: 5805, 35: 4084, 36: 5525, 37: 6265, 38: 239, 39: 20, 40: 3322, 41: 47, 42: 51, 43: 6, 44: 146, 46: 1046, 47: 10, 48: 190, 49: 36, 50: 23, 51: 1025, 52: 25, 53: 113, 54: 223, 55: 126, 56: 24, 57: 10, 58: 8705, 59: 4, 60: 242, 61: 1134, 62: 3000, 63: 1987, 64: 49, 65: 5309, 66: 5000}
66


In [5]:
for i, (sport, athlete_urls) in enumerate(sport_athletes.items(),start=1):

    if i<max_sport_index:
        continue

    all_dicts=[]
    for j, athlete_url in enumerate(athlete_urls,start=1):
        
        if i in max_scraped:
            athlete_index= max_scraped[i]
            if j<=athlete_index:
                continue

        try:
            html=requests.get(athlete_url,headers).content
            soup=BeautifulSoup(html,'html.parser')

            bio_dict={e.th.text:e.td.text for e in soup.select('table[class="biodata"] tr')}
            bio_dict['Sport']=sport
            bio_dict['URL']=athlete_url
            
            all_dicts.append(bio_dict)

        except:
            continue

        print(f'Sport: {i}/{len(sport_athletes)}, Athlete: {j}/{len(athlete_urls)}'.ljust(100),end='\r')

        if j%1000==0 or j==len(athlete_urls):
            pickle.dump(all_dicts, open(f'scraped_athletes/all_dicts_{i}_{j}.pickle','wb'))
            all_dicts=[]
